# 과제 :KNN classifier 구현 
1) Preprocessing / EDA  
2) KNN classifier
- K
- Distance Measure : Euclidean? Manhattan?
- Weighted voting? Majority voting?
  
3) Evaluation

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsClassifier
import math

In [2]:
iris = load_iris()

In [3]:
#iris.data -> features, iris.target -> labels
X = iris.data
y = iris.target

- About data : https://www.kaggle.com/uciml/iris

In [4]:
# split into test and train dataset, and use random_state=48
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=48)

In [6]:
trainSet=np.column_stack((X_train, y_train))
testSet=np.column_stack((X_test, y_test))

# KNN 구현

In [7]:
from scipy import spatial

In [8]:
# p=1 : return Manhattan Distance
# p=2 : return Eucludean Distance
# w=1 : Normal
# w=2 : weighted
def KNN(p,trainSet, testSet, k, w):
    acc=0
    for i in range(len(testSet)):
        indexing=[]
        distance=[]
        
        if p ==1:
            for q in range(len(trainSet)):
                indexing.append(q)
                distance.append(abs(trainSet[q,:4]-testSet[i,:4]).sum())
        elif p ==2 :
            for q in range(len(trainSet)):
                indexing.append(q)
                distance.append(np.sqrt(((trainSet[q,:4]-testSet[i,:4])**2).sum()))
                
        neighbor=pd.Series(distance,indexing)
        neighbor.sort_values(inplace=True)
        neighbor=neighbor.iloc[:k]
        
        if w ==1 :    
            pred= pd.DataFrame(trainSet).iloc[list(neighbor.index.astype(int)),-1].mode()[0]
        elif w==2 :
            pred_df=pd.concat([np.reciprocal(neighbor)/(np.reciprocal(neighbor).sum()),pd.DataFrame(trainSet).iloc[list(neighbor.index.astype(int)),-1]],axis=1)
            pred_df.columns=['weight','category']
            pred= (pred_df.groupby('category').sum().reset_index().sort_values(by='weight',ascending=False)).iloc[0,0]
            
        
        print(str(i) + ' > predicted : ' + str(pred) + ', actual : ' + str(testSet[i][-1]))
        if pred == testSet[i][-1]:
            acc+=1
    print('Accuracy: ' + str(acc/len(testSet)) + '%')

In [9]:
KNN(1,trainSet,testSet,3,1)
# p=1 : return Manhattan Distance
# p=2 : return Eucludean Distance
# w=1 : Normal
# w=2 : weighted

0 > predicted : 1.0, actual : 1.0
1 > predicted : 1.0, actual : 1.0
2 > predicted : 2.0, actual : 2.0
3 > predicted : 0.0, actual : 0.0
4 > predicted : 1.0, actual : 1.0
5 > predicted : 2.0, actual : 2.0
6 > predicted : 0.0, actual : 0.0
7 > predicted : 1.0, actual : 2.0
8 > predicted : 0.0, actual : 0.0
9 > predicted : 1.0, actual : 1.0
10 > predicted : 2.0, actual : 2.0
11 > predicted : 0.0, actual : 0.0
12 > predicted : 0.0, actual : 0.0
13 > predicted : 2.0, actual : 2.0
14 > predicted : 1.0, actual : 1.0
15 > predicted : 1.0, actual : 1.0
16 > predicted : 0.0, actual : 0.0
17 > predicted : 1.0, actual : 1.0
18 > predicted : 2.0, actual : 1.0
19 > predicted : 2.0, actual : 2.0
20 > predicted : 0.0, actual : 0.0
21 > predicted : 2.0, actual : 2.0
22 > predicted : 1.0, actual : 1.0
23 > predicted : 1.0, actual : 1.0
24 > predicted : 2.0, actual : 2.0
25 > predicted : 0.0, actual : 0.0
26 > predicted : 0.0, actual : 0.0
27 > predicted : 2.0, actual : 2.0
28 > predicted : 2.0, actual :

In [10]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
neigh = KNeighborsClassifier(n_neighbors=3,p=1)
neigh.fit(trainSet[:,:4],trainSet[:,-1])
pred=neigh.predict(testSet[:,:4])
display(accuracy_score(pred,testSet[:,4])) # 동일

0.9111111111111111

In [11]:
KNN(2,trainSet,testSet,3,1)

0 > predicted : 1.0, actual : 1.0
1 > predicted : 1.0, actual : 1.0
2 > predicted : 2.0, actual : 2.0
3 > predicted : 0.0, actual : 0.0
4 > predicted : 1.0, actual : 1.0
5 > predicted : 2.0, actual : 2.0
6 > predicted : 0.0, actual : 0.0
7 > predicted : 1.0, actual : 2.0
8 > predicted : 0.0, actual : 0.0
9 > predicted : 1.0, actual : 1.0
10 > predicted : 2.0, actual : 2.0
11 > predicted : 0.0, actual : 0.0
12 > predicted : 0.0, actual : 0.0
13 > predicted : 2.0, actual : 2.0
14 > predicted : 1.0, actual : 1.0
15 > predicted : 1.0, actual : 1.0
16 > predicted : 0.0, actual : 0.0
17 > predicted : 1.0, actual : 1.0
18 > predicted : 2.0, actual : 1.0
19 > predicted : 2.0, actual : 2.0
20 > predicted : 0.0, actual : 0.0
21 > predicted : 2.0, actual : 2.0
22 > predicted : 1.0, actual : 1.0
23 > predicted : 1.0, actual : 1.0
24 > predicted : 2.0, actual : 2.0
25 > predicted : 0.0, actual : 0.0
26 > predicted : 0.0, actual : 0.0
27 > predicted : 2.0, actual : 2.0
28 > predicted : 2.0, actual :

In [12]:
neigh = KNeighborsClassifier(n_neighbors=3,p=2)
neigh.fit(trainSet[:,:4],trainSet[:,-1])
pred=neigh.predict(testSet[:,:4])
display(accuracy_score(pred,testSet[:,4])) # 동일

0.9555555555555556

# HyperParameter Searching ( GridSearch )

In [13]:
dis_list=[1,2]
num_k_list=[3,5,7,9]
weight_list=[1,2]

In [14]:
from itertools import product
items = [dis_list,num_k_list,weight_list]
params=list(product(*items))

In [15]:
scores=[]
dis=[]
num_k=[]
weight=[]
def tune_KNN(p,trainSet, testSet, k, w):
    acc=0
    for i in range(len(testSet)):
        indexing=[]
        distance=[]
        
        if p ==1:
            for q in range(len(trainSet)):
                indexing.append(q)
                distance.append(abs(trainSet[q,:4]-testSet[i,:4]).sum())
        elif p ==2 :
            for q in range(len(trainSet)):
                indexing.append(q)
                distance.append(np.sqrt(((trainSet[q,:4]-testSet[i,:4])**2).sum()))
                
        neighbor=pd.Series(distance,indexing)
        neighbor.sort_values(inplace=True)
        neighbor=neighbor.iloc[:k]
        
        if w ==1 :    
            pred= pd.DataFrame(trainSet).iloc[list(neighbor.index.astype(int)),-1].mode()[0]
        elif w==2 :
            pred_df=pd.concat([np.reciprocal(neighbor)/(np.reciprocal(neighbor).sum()),pd.DataFrame(trainSet).iloc[list(neighbor.index.astype(int)),-1]],axis=1)
            pred_df.columns=['weight','category']
            pred= (pred_df.groupby('category').sum().reset_index().sort_values(by='weight',ascending=False)).iloc[0,0]
            
        
        
        if pred == testSet[i][-1]:
            acc+=1
    dis.append(p)
    num_k.append(k)
    weight.append(w)
    scores.append(acc/len(testSet))

In [16]:
for i in params:
    p,k,w=i
    tune_KNN(p,trainSet,testSet,k,w)

In [17]:
score=pd.DataFrame([dis,num_k,weight,scores]).T

In [18]:
score.columns=['d','k','w','score']

In [19]:
score.sort_values(by='score',ascending=False)
# 각 경우의 수별 score

,d,k,w,score
14,2.0,9.0,1.0,1.000000
13,2.0,7.0,2.0,0.977778
15,2.0,9.0,2.0,0.977778
6,1.0,9.0,1.0,0.955556
8,2.0,3.0,1.0,0.955556
9,2.0,3.0,2.0,0.955556
10,2.0,5.0,1.0,0.955556
11,2.0,5.0,2.0,0.955556
12,2.0,7.0,1.0,0.955556
0,1.0,3.0,1.0,0.933333
